# pytorch를 라이브러리로 Neural Networks 초음파 이미지 분류 - 이승연

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

In [21]:
# 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지 정규화
])
train_data = datasets.ImageFolder('./deep learnig/train', transform=transform)
val_data = datasets.ImageFolder('./deep learnig/val', transform=transform)

# 데이터 로더
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False)

# 모델 아키텍처 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(150528, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = x.view(-1, 150528)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = Net()

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# 모델 학습
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # 검증 데이터를 사용하여 모델 정확도 평가
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Accuracy: {correct / total}")

Epoch 1, Loss: 1.325530907464406, Accuracy: 0.37859424920127793
Epoch 2, Loss: 1.202498579782153, Accuracy: 0.4728434504792332
Epoch 3, Loss: 1.127436743369178, Accuracy: 0.4057507987220447
Epoch 4, Loss: 1.0827309900806064, Accuracy: 0.4904153354632588
Epoch 5, Loss: 1.0528858824381753, Accuracy: 0.5255591054313099
Epoch 6, Loss: 1.0419121923900785, Accuracy: 0.46485623003194887
Epoch 7, Loss: 1.014709006226252, Accuracy: 0.41373801916932906
Epoch 8, Loss: 1.0094826453261905, Accuracy: 0.4680511182108626
Epoch 9, Loss: 0.9991885787910886, Accuracy: 0.5191693290734825
Epoch 10, Loss: 0.9869623983663226, Accuracy: 0.36261980830670926


### loss가 1보다 높아 해결 방법 조사

#### batch size 높이기, lr 낮추기(0.0001 로 변경), loss=MSE로 변경. layer추가

In [ ]:
# 그 전 코드

# # 데이터셋 로드 및 전처리
# transform = transforms.Compose([
#     transforms.Resize((224, 224)),  # 이미지 크기 조정
#     transforms.ToTensor(),  # 이미지를 텐서로 변환
#     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지 정규화
# ])
# train_data = datasets.ImageFolder('./deep learnig/train', transform=transform)
# val_data = datasets.ImageFolder('./deep learnig/val', transform=transform)

# # 데이터 로더
# train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
# val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False)

# # 모델 아키텍처 정의
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         self.conv1 = nn.Conv2d(3, 6, 5)
#         self.pool = nn.MaxPool2d(2, 2)
#         self.conv2 = nn.Conv2d(6, 16, 5)
#         self.fc1 = nn.Linear(150528, 120) # self.fc1 = nn.Linear(16 * 5 * 5, 120)
#         self.fc2 = nn.Linear(120, 84)
#         self.fc3 = nn.Linear(84, 4)

#     def forward(self, x):
#         # x = self.pool(nn.functional.relu(self.conv1(x)))
#         # x = self.pool(nn.functional.relu(self.conv2(x)))
#         x = x.view(-1, 150528) # x = x.view(-1, 16 * 5 * 5)    # 150528 = 224 * 224 * 3
#         x = nn.functional.relu(self.fc1(x))
#         x = nn.functional.relu(self.fc2(x))
#         x = self.fc3(x)
#         return x

# # 모델 초기화
# model = Net()

# # 손실 함수와 최적화 알고리즘 정의
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# # 모델 학습
# for epoch in range(10):
#     running_loss = 0.0
#     for i, data in enumerate(train_loader, 0):
#         inputs, labels = data

#         optimizer.zero_grad()

#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()

#         running_loss += loss.item()

#     # 검증 데이터를 사용하여 모델 정확도 평가
#     correct = 0
#     total = 0
#     with torch.no_grad():
#         for data in val_loader:
#             images, labels = data
#             outputs = model(images)
#             _, predicted = torch.max(outputs.data, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Accuracy: {correct / total}")

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as datasets

# 데이터셋 로드 및 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # 이미지 크기 조정
    transforms.ToTensor(),  # 이미지를 텐서로 변환
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # 이미지 정규화
])
train_data = datasets.ImageFolder('./deep learnig/train', transform=transform)
val_data = datasets.ImageFolder('./deep learnig/val', transform=transform)

# 데이터 로더
train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False)

# 모델 아키텍처 정의
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(224 * 224 * 3, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 4)

    def forward(self, x):
        x = x.view(-1, 224 * 224 * 3)
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x

# 모델 초기화
model = Net()

# 손실 함수와 최적화 알고리즘 정의
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.0001, momentum=0.9)

# 모델 학습
for epoch in range(10):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # 검증 데이터를 사용하여 모델 정확도 평가
    correct = 0
    total = 0
    with torch.no_grad():
        for data in val_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f"Epoch {epoch + 1}, Loss: {running_loss / len(train_loader)}, Accuracy: {correct / total}")
